In [ ]:
import ee
ee.Initialize()

In [ ]:
import geemap
from pprint import pprint
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import shapely as shp
import os
import re
import string
import math

In [ ]:
def geometryToEE(img_geometry):
    
    # format geometry
#     img_geometry = gpd.GeoDataFrame(geometry = img_geometry)
    print(img_geometry)
    img_geometry = img_geometry.reset_index(drop = True)
    img_geometry = [[[x, y] for x, y in list(img_geometry.geometry.exterior[0].coords)]]
    
    # convert geometry to ee.Geometry
    img_geometry_ee = ee.Geometry({
        'type': 'Polygon',
        'coordinates': img_geometry
    })
    
    return img_geometry_ee

In [ ]:
# Read in RTS test tiles
tiles = gpd.read_file('../data/rts_polygons/all_tiles.shp')
tiles = tiles.to_crs(4326)

aoi = tiles.geometry.bounds.iloc[0]
print(aoi)
aoi = ee.Geometry({
        'type': 'Polygon',
        'coordinates': [
            [
                [aoi[0], aoi[2]],
                [aoi[1], aoi[2]],
                [aoi[1], aoi[3]],
                [aoi[0], aoi[3]],
                [aoi[0], aoi[2]]
                        ]
        ]
    })
print(aoi.getInfo())

tiles

In [ ]:
# Prep Map
Map = geemap.Map()
Map.centerObject(aoi);

In [ ]:
# Get Sentinel-2 Mosaic to Align Planet Data
MAX_CLOUD_PROBABILITY = 50
# geometry = ee.Geometry({
#     'type': 'Polygon',
#     'coordinates': [[
#         [66, 66],
#         [66, 73.5],
#         [85, 73.5],
#         [85, 66],
#         [66, 66]
#         ]]
# })

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
#     .filterBounds(geometry)
    .filterDate('2018-06-15', '2019-08-31')
    .filter(ee.Filter.dayOfYear(182, 243))
);

s2_clouds = (
    ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
#     .filterBounds(geometry)
    .filterDate('2018-06-15', '2019-08-31')
    .filter(ee.Filter.dayOfYear(182, 243))
);

def s2_maskcloud(image):
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    return image.updateMask(isNotCloud)

# Join S2 SR with cloud probability dataset to add cloud mask.
s2_with_clouds = ee.Join.saveFirst('cloud_mask').apply(    
  primary=s2,
  secondary=s2_clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

s2_masked = ee.ImageCollection(s2_with_clouds).map(s2_maskcloud)

# s2_masked_north = s2_masked.filter(ee.Filter.dayOfYear(196, 243))

def s2_water_mask(image):
    return image.select('SCL').eq(6)

s2_water_median = (
    s2_masked.map(s2_water_mask)
    .median()
);

s2_water_max = (
    s2_masked.map(s2_water_mask)
    .max()
);
s2_water = s2_water_median.addBands(s2_water_max)

s2_mosaic = (
    s2_masked
    .select(['B2', 'B3', 'B4', 'B8'])
    .median());

# s2_mosaic_north = (
#     s2_masked_north
#     .select(['B2', 'B3', 'B4', 'B8'])
#     .median());

pprint(s2_water.getInfo())

In [ ]:
Map.addLayer(s2_mosaic.clip(aoi), 
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1000}, 
             'S2 All Years')
Map.addLayer(s2_water_median.clip(aoi), 
             {'min':0, 'max':1}, 
             'S2 Water (Median)')
Map.addLayer(s2_water_max.clip(aoi), 
             {'min':0, 'max':1}, 
             'S2 Water (Max)')
Map.addLayer(s2_water.clip(aoi), 
             {'min':0, 'max':1}, 
             'S2 Water')

In [ ]:
Map

In [ ]:
for index, row in tiles.iterrows():
    polygon_id = row.id
    name = 'Sentinel-2_water_mask_' + row.imagery + str(row.id)
    img_geometry = [[[x, y] for x, y in list(row.geometry.exterior.coords)]]
    crs = 'EPSG:' + str(3413)
   
    # Export 2018-2019 growing season Sentinel-2 Mosaic to Drive
    task = ee.batch.Export.image.toDrive(
        image = s2_water.toFloat(),
        description = name,
        folder = 'Earth Engine Exports',
        crs = crs,
        scale = 10,
        maxPixels = 1e13,
        region = img_geometry,
        fileFormat = 'GeoTIFF'
    )
    task.start()